In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Split values based on the two CSVs
training_csv = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv")
testing_csv = pd.read_csv("http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv")

#training_csv.to_csv('titanic_training.csv', index=False, encoding='utf-8')
#testing_csv.to_csv('titanic_testing.csv', index=False, encoding='utf-8')

data = training_csv
target = data['Survived'] 

In [3]:
# Split values into numerical and categorical features
numerical_features = data[['Fare','Pclass','Age']]
# Need to handle values where Age is missing
features = pd.concat([data[['Fare', 'Pclass', 'Age']],
                           pd.get_dummies(data['Sex'], prefix='Sex'),
                           pd.get_dummies(data['Embarked'], prefix='Embarked')],
                          axis=1)
features = features.drop('Sex_male', 1)
features.head(5)

,Fare,Pclass,Age,Sex_female,Embarked_C,Embarked_Q,Embarked_S
0,7.2500,3,22,0,0,0,1
1,71.2833,1,38,1,1,0,0
2,7.9250,3,26,1,0,0,1
3,53.1000,1,35,1,0,0,1
4,8.0500,3,35,0,0,0,1


In [4]:
# We will try to loop through the different classes and sex to apply the median age
for sex in xrange(0,2):
    for cabin_class in xrange(0,4):
        median_age_group = features[(features.Pclass == cabin_class) & (features.Sex_female == sex)].dropna().median()
        features[(features.Pclass == cabin_class) 
                 & (features.Sex_female == sex)] = features[(features.Pclass == cabin_class) 
                                                                     & (features.Sex_female == sex)
                                                                    ].fillna(median_age_group)
# adding validation for if the person is a child
features['child'] = (features.Age <= 18)*1

In [8]:
features.head()

,Fare,Pclass,Age,Sex_female,Embarked_C,Embarked_Q,Embarked_S,child
0,7.2500,3,22,0,0,0,1,0
1,71.2833,1,38,1,1,0,0,0
2,7.9250,3,26,1,0,0,1,0
3,53.1000,1,35,1,0,0,1,0
4,8.0500,3,35,0,0,0,1,0


In [5]:
# Build Model

In [6]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline

In [9]:
X = np.asarray(features.values, 'float32')
X

array([[  7.25      ,   3.        ,  22.        , ...,   0.        ,
          1.        ,   0.        ],
       [ 71.28330231,   1.        ,  38.        , ...,   0.        ,
          0.        ,   0.        ],
       [  7.92500019,   3.        ,  26.        , ...,   0.        ,
          1.        ,   0.        ],
       ..., 
       [ 23.45000076,   3.        ,  21.5       , ...,   0.        ,
          1.        ,   0.        ],
       [ 30.        ,   1.        ,  26.        , ...,   0.        ,
          0.        ,   0.        ],
       [  7.75      ,   3.        ,  32.        , ...,   1.        ,
          0.        ,   0.        ]], dtype=float32)

In [12]:
target.values

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1,

In [15]:
def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((8, 8)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

# Load Data
#digits = datasets.load_digits()
X = np.asarray(features.values, 'float32')
Y = target
#X, Y = nudge_dataset(X, target)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=0)

# Models we will use
logistic = linear_model.LogisticRegression()
rbm = BernoulliRBM(random_state=0, verbose=True)

classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

###############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000.0

# Training RBM-Logistic Pipeline
classifier.fit(X_train, Y_train)

# Training Logistic regression
logistic_classifier = linear_model.LogisticRegression(C=100.0)
logistic_classifier.fit(X_train, Y_train)

###############################################################################
# Evaluation

print()
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        classifier.predict(X_test))))

[BernoulliRBM] Iteration 1, pseudo-likelihood = -7.30, time = 0.06s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -5.73, time = 0.04s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -4.63, time = 0.03s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -4.79, time = 0.03s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -4.77, time = 0.04s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -4.48, time = 0.05s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -4.86, time = 0.07s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -4.19, time = 0.06s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -4.22, time = 0.05s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -4.17, time = 0.07s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -4.14, time = 0.07s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -4.22, time = 0.04s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -3.84, time = 0.04s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -4.09, time = 0.06s
[BernoulliRBM] Iteration 15, pseudo-likelih

In [7]:
from __future__ import print_function

print(__doc__)

# Authors: Yann N. Dauphin, Vlad Niculae, Gabriel Synnaeve
# License: BSD

import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline


###############################################################################
# Setting up

def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((8, 8)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

# Load Data
digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
X, Y = nudge_dataset(X, digits.target)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=0)

# Models we will use
logistic = linear_model.LogisticRegression()
rbm = BernoulliRBM(random_state=0, verbose=True)

classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

###############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000.0

# Training RBM-Logistic Pipeline
classifier.fit(X_train, Y_train)

# Training Logistic regression
logistic_classifier = linear_model.LogisticRegression(C=100.0)
logistic_classifier.fit(X_train, Y_train)

###############################################################################
# Evaluation

print()
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        classifier.predict(X_test))))

print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(
        Y_test,
        logistic_classifier.predict(X_test))))

Automatically created module for IPython interactive environment
[BernoulliRBM] Iteration 1, pseudo-likelihood = -25.39, time = 0.22s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -23.77, time = 0.42s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -22.94, time = 0.36s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -21.91, time = 0.34s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -21.69, time = 0.42s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -21.06, time = 0.47s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -20.89, time = 0.48s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -20.64, time = 0.41s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -20.36, time = 0.34s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -20.09, time = 0.33s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -20.08, time = 0.35s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -19.82, time = 0.38s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -19.64, time = 0.58s
[BernoulliRBM] Iteration 14, pseud